In [154]:
import networkx as nx
import numpy as np
import cv2
import json
import networkx as nx
import re
import argparse
import glob
import os

from route_generator import iter_all_white_points, draw_waypoint, rdp_algorithm, recorded_find_path
from add_path_to_graph import add_path, read_adjlist_with_tuples

##################### Input Town number #####################

town = int(input("Enter town number: 1, 2, 3, or 10"))

if town == 10:
    waypoint_map = cv2.imread("./waypoint_maps/Town10HD.png", 1)
    town_name = "Town10HD"
else:
    town_name = f"Town0{town}"
    waypoint_map = cv2.imread(f"./waypoint_maps/{town_name}.png", 1)


##################### Read latest graph list of the given town #####################

list_of_files = glob.glob(f'graph_list/{town_name}/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)

if town == 10:
    index = int(latest_file[30:-8])
    new_file_name = latest_file[:30] + str(index+1) + latest_file[-8:]
else:
    index = int(latest_file[27:-8])
    new_file_name = latest_file[:27] + str(index+1) + latest_file[-8:]

G = read_adjlist_with_tuples(latest_file)
print(latest_file)

graph_list/Town10HD\10_graph_v61.adjlist


In [157]:
################# Change start, end point #################

reverse = input("Do reverse search or not? (y for reverse search, other for original search)\n")

start, end = (2325, 1170), (2321, 1146)

if reverse == "y":
    start, end = end, start


In [158]:
if start == (0,0) or end == (0,0):
    print("Didn't change to your point!!!!!!!!")

all_whites_pos = iter_all_white_points(waypoint_map)

new_img = waypoint_map.copy()
new_img[start[0]][start[1]] = np.array([0, 0, 255])
new_img[end[0]][end[1]] = np.array([0, 0, 255])

cv2.circle(new_img, start[::-1], radius=5, color=(255, 0, 0), thickness=5)
cv2.circle(new_img, end[::-1], radius=5, color=(0, 255, 0), thickness=5)


print("Start point:", start)
print("End point:", end)
print("Finding Path...")
# final_path = find_path(start, end, all_whites_pos)
flag, final_path = recorded_find_path(start, end, all_whites_pos, "", True)

##### Path not found #####
if flag == False:
    print("Not found!!!")
    print(final_path)
    for i, j in final_path[0]:
        new_img[i][j] = np.array([255, 255, 0])
    
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/not_found/test__{start}_{end}.png", new_img)
else:
    print("found!!!")
    print(final_path[0])
    print("Adding to the graph list...")
    
    if reverse == "y":
        fp = final_path[0][::-1]
        fp = [fp]
        add_path(G, fp, new_file_name)
    else:
        add_path(G, final_path, new_file_name)
    
    print(new_file_name, "Store!")
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/found/test__{start}_{end}.png", rdp_img)

Start point: (2323, 866)
End point: (2127, 534)
Finding Path...
Get!!!
found!!!
[(2323, 866), (2323, 862), (2323, 858), (2323, 855), (2323, 851), (2323, 848), (2323, 844), (2322, 841), (2317, 840), (2317, 836), (2317, 833), (2318, 829), (2321, 823), (2318, 822), (2320, 816), (2318, 814), (2319, 809), (2318, 807), (2319, 806), (2317, 800), (2318, 799), (2316, 793), (2317, 792), (2316, 789), (2315, 785), (2314, 782), (2314, 778), (2313, 775), (2312, 772), (2311, 768), (2310, 765), (2309, 761), (2307, 755), (2306, 751), (2304, 745), (2303, 741), (2301, 735), (2300, 731), (2297, 725), (2296, 722), (2294, 718), (2292, 712), (2289, 706), (2287, 702), (2284, 696), (2281, 690), (2278, 684), (2276, 681), (2274, 678), (2271, 672), (2269, 669), (2267, 666), (2265, 663), (2263, 660), (2261, 657), (2258, 651), (2256, 648), (2254, 645), (2251, 642), (2249, 640), (2247, 637), (2245, 634), (2243, 631), (2239, 626), (2236, 623), (2234, 621), (2232, 618), (2229, 615), (2227, 613), (2225, 610), (2222, 60

In [46]:
############### Quick test for other start point with the same goal point
start, end = (3664, 2052), (3997, 2755)

fp = nx.shortest_path(G,  start,  end)
print(fp)

NodeNotFound: Either source (3664, 2052) or target (3997, 2755) is not in G